In [1]:
import gempipe
import cobra

# Testing reference GSMM

Before preceeding with `gempipe recon`

In [2]:
Lreuteri_530 = cobra.io.read_sbml_model('reference/Lreuteri_530.xml')

In [3]:
gempipe.sanity_report(Lreuteri_530)

gempipe v1.37.7 - sanity_report
model ID: LbReuteri
G: 530 R: 714 M: 660 uM: 562 groups: 0
Compartments: ['', 'c', 'e']
Biomass assemblies: 1 ['BIOMASS']
Objectives: 1 ['BIOMASS']
Optimization: 0.7265808565661749 optimal (cplex)
Unconstrained LB-UB: (-1000.0, 1000.0)
Bad EX_change notation: 0
Sinks/demands: 3
Constrained metabolic: 3
With 'artificial' atoms: 88
Missing formulas - charges: 21 - 0
Mass - charge unbalances: 6 - 7


In [4]:
# there are 61 reactions with 'None' as GPR. 
# there is no distinction between orphans and spontaneous.
# 'None' has been modeled as a string. 
# Remove the gene 'None':
cnt = 0
for r in Lreuteri_530.reactions:  # 61
    if r.gene_reaction_rule == 'None': #and r.id.startswith('EX_')==False: 
        r.gene_reaction_rule = ''
        r.update_genes_from_gpr()
        cnt += 1
        #print(cnt, '\t', r.id, '\n\t', r.name, '\n\t', r.reaction)
cobra.manipulation.delete.remove_genes(Lreuteri_530, ['None'], remove_reactions=False)

In [5]:
# there is a strange compartment (''), fix here:
for m in Lreuteri_530.metabolites: 
    if m.compartment == '': 
        if m.id.endswith('_e'): m.compartment = 'e'
        if m.id.endswith('_c'): m.compartment = 'c'

In [6]:
# the model has a demand that is not needed for growth in these conditions: 
gempipe.check_sinks_demands(Lreuteri_530)
with Lreuteri_530: 
    res = Lreuteri_530.optimize()
    print(res.objective_value, res.status)
    Lreuteri_530.reactions.DM_2ahbut_c.bounds = (0, 0)
    res = Lreuteri_530.optimize()
    print(res.objective_value, res.status)

1 : DM_2ahbut_c : 2ahbut_c -->  : (0.0, 1000.0)
0.7265808565661749 optimal
0.7265808565661749 optimal


In [7]:
_ = gempipe.check_constrained_metabolic(Lreuteri_530)

1 : TDPDRR : (-1000.0, 0.0) : dtdprmn_c + nadp_c <-- dtdp4d6dm_c + h_c + nadph_c
2 : ATPM : (0.36, 0.36) : atp_c + h2o_c --> adp_c + h_c + pi_c
3 : PFK : (-1000.0, 2.0) : atp_c + f6p_c <=> adp_c + fdp_c + h_c


In [8]:
_ = gempipe.check_artificial_atoms(Lreuteri_530, preview=3)

1 : X : but2eACP_c[C4H4OX], 2tddacp_c[C12H20OX], 2tdeacp_c[C10H16OX], ... (46 in total)
2 : M : MCOOH_c[MCO2], MCOSH_c[MCHOS]
3 : R : alatrna_c[C3H6NOR], argtrna_c[C6H14N4OR], asntrna_c[C14H24N2O12PR2], ... (40 in total)


In [9]:
_ = gempipe.check_mass_unbalances(Lreuteri_530)

1 : DAGK_LRE : atp_c + dag_LRE_c --> adp_c + h_c + pa_LRE_c : {'H': 1.0}
2 : PGPP_LRE : h2o_c + pgp_LRE_c --> pg_LRE_c + pi_c : {'P': 1.0, 'O': 3.0}
3 : CLPNS_LRE : 2.0 pg_LRE_c <=> clpn_LRE_c + glyc_c : {'C': 3.0, 'O': 3.0, 'H': 8.0}
4 : PGSA_LRE : cdpdag_LRE_c + glyc3p_c <=> cmp_c + h_c + pgp_LRE_c : {'C': 6.0, 'O': 2.0, 'N': 3.0, 'H': 6.0}
5 : PPOHt : ppoh_e <=> ppoh_c : {'C': 3.0, 'O': 1.0, 'H': 8.0}
6 : PAP_LRE : h2o_c + pa_LRE_c --> dag_LRE_c + pi_c : {'P': 1.0, 'O': 3.0}


In [10]:
_ =  gempipe.check_charge_unbalances(Lreuteri_530)

1 : DAGK_LRE : atp_c + dag_LRE_c --> adp_c + h_c + pa_LRE_c : {'charge': 2.0}
2 : PGGT4 : uaagmdalac_c --> PGlac2_c + udcpdp_c : {'charge': 1.0}
3 : RNAS_LRE : 0.710810743235 atp_c + 0.187747387186 ctp_c + 0.21245220899 gtp_c + 0.288989660589 utp_c --> RNA_LRE_c + 0.4 adp_c + 0.4 h_c + 0.4 pi_c + ppi_c : {'charge': 1.0}
4 : PGSA_LRE : cdpdag_LRE_c + glyc3p_c <=> cmp_c + h_c + pgp_LRE_c : {'charge': 1.0}
5 : GAT1_LRE : 0.0211 cpocdacp_c + glyc3p_c + 0.0258 hdeACP_c + 0.075 ocdacp_c + 0.0418 ocdctrACP_c + 0.3519 ocdcyaACP_c + 0.1591 octeACP_c + 0.2566 palmACP_c + 0.0687 tdeacp_c --> ACP_c + agly3p_LRE_c : {'charge': 2.0000000000000004}
6 : DNAS_LRE : 1.37 atp_c + 0.310284595735 datp_c + 0.180906354753 dctp_c + 0.207484087701 dgtp_c + 0.301324961811 dttp_c + 1.37 h2o_c --> DNA_LRE_c + 1.37 adp_c + 1.37 h_c + 1.37 pi_c + ppi_c : {'charge': 0.9999999999999982}
7 : CPSS_LRE : 4.0 h2o_c + 4.0 udpg_c <=> CPS_LRE_c + 5.0 h_c + 3.0 udp_c + ump_c : {'charge': 2.0}


In [11]:
with Lreuteri_530: 
    Lreuteri_530.reactions.ATPM.bounds = (0,0)
    gempipe.verify_egc_all(Lreuteri_530)

unknown metabolite 'fadh2_c' created
No energy-generating cycles (EGCs) found.


# Define growth medium

In [12]:
# constrained EXR: 5 aa + glucose
# 1 missing aa (glutamine)

for r in Lreuteri_530.reactions: 
    if r.id.startswith('EX_') and r.lower_bound == -1000: 
        print(r.id,'\t',  r.name, '\t', r.bounds)

EX_ura_e 	 Exchange Uracil 	 (-1000.0, 1000.0)
EX_h2o_e 	 Exchange H2O 	 (-1000.0, 1000.0)
EX_na1_e 	 Exchange Sodium 	 (-1000.0, 1000.0)
EX_pnto__R_e 	 Exchange (R)-Pantothenate 	 (-1000.0, 1000.0)
EX_val__L_e 	 Exchange L-Valine 	 (-1000.0, 1000.0)
EX_ade_e 	 Exchange Adenine 	 (-1000.0, 1000.0)
EX_cys__L_e 	 Exchange L-Cysteine 	 (-1000.0, 1000.0)
EX_mn2_e 	 Exchange Mn2+ 	 (-1000.0, 1000.0)
EX_btn_e 	 Exchange Biotin 	 (-1000.0, 1000.0)
EX_his__L_e 	 Exchange L-Histidine 	 (-1000.0, 1000.0)
EX_h_e 	 Exchange H+ 	 (-1000.0, 1000.0)
EX_gua_e 	 Exchange Guanine 	 (-1000.0, 1000.0)
EX_gly_e 	 Exchange Glycine 	 (-1000.0, 1000.0)
EX_k_e 	 Exchange K+ 	 (-1000.0, 1000.0)
EX_leu__L_e 	 Exchange L-Leucine 	 (-1000.0, 1000.0)
EX_pi_e 	 Exchange Phosphate 	 (-1000.0, 1000.0)
EX_4abz_e 	 Exchange 4-Aminobenzoate 	 (-1000.0, 1000.0)
EX_phe__L_e 	 Exchange L-Phenylalanine 	 (-1000.0, 1000.0)
EX_pydam_e 	 Exchange Pyridoxamine 	 (-1000.0, 1000.0)
EX_ala__L_e 	 Exchange L-Alanine 	 (-1000.0, 1000

In [13]:
for r in Lreuteri_530.reactions: 
    if r.id.startswith('EX_') and r.lower_bound < 0 and r.lower_bound > -1000: 
        print(r.id,'\t',  r.name, '\t', r.bounds)

EX_asp__L_e 	 Exchange L-Aspartate 	 (-0.29, 1000.0)
EX_arg__L_e 	 Exchange L-Arginine 	 (-0.42, 1000.0)
EX_glc__D_e 	 Exchange D-Glucose 	 (-25.2, 1000.0)
EX_ser__L_e 	 Exchange L-Serine 	 (-0.34, 1000.0)
EX_glu__L_e 	 Exchange L-Glutamate 	 (-0.47, 1000.0)
EX_asn__L_e 	 Aspargine exchange 	 (-0.23, 1000.0)


In [14]:
for r in Lreuteri_530.reactions: 
    if r.id.startswith('EX_') and r.upper_bound < 1000: 
        print(r.id,'\t',  r.name, '\t', r.bounds)

EX_btd_RR_e 	 Exchange (R,R)-2,3-Butanediol 	 (0.0, 0.0)
EX_ac_e 	 Exchange Acetate 	 (0.0, 0.0)
EX_lac__D_e 	 Exchange D-Lactate 	 (0.0, 11.74)
EX_lac__L_e 	 Exchange L-Lactate 	 (0.0, 11.74)
EX_etoh_e 	 Exchange Ethanol 	 (0.0, 24.4)
EX_13ppd_e 	 Exchange Propane-1,3-diol 	 (0.0, 0.0)


In [15]:

res = Lreuteri_530.optimize()
print(res.objective_value, res.status)

# relax secretion exchanges
Lreuteri_530.reactions.EX_btd_RR_e.bounds = (0, 1000)
Lreuteri_530.reactions.EX_ac_e.bounds = (0, 1000)
Lreuteri_530.reactions.EX_lac__D_e.bounds = (0, 1000)
Lreuteri_530.reactions.EX_lac__L_e.bounds = (0, 1000)
Lreuteri_530.reactions.EX_etoh_e.bounds = (0, 1000)
Lreuteri_530.reactions.EX_13ppd_e.bounds = (0, 1000)

res = Lreuteri_530.optimize()
print(res.objective_value, res.status)

for r in Lreuteri_530.reactions: 
    if r.id.startswith("EX_"): 
        if r.lower_bound < 0: 
            r.lower_bound = -5

res = Lreuteri_530.optimize()
print(res.objective_value, res.status)

Lreuteri_530.reactions.EX_glc__D_e.bounds = (-10, 0)

res = Lreuteri_530.optimize()
print(res.objective_value, res.status)
            

0.7265808565661712 optimal
0.7443757243761258 optimal
0.39096374766411396 optimal
0.5650348730711057 optimal


In [16]:
# verify growth in absence of glucose (useful for defing the --minflux parameter)

# check growth without glucose: 
with Lreuteri_530: 
    res = Lreuteri_530.optimize()
    print(res.objective_value, res.status)
    Lreuteri_530.reactions.EX_glc__D_e.bounds = (0, 0)
    res = Lreuteri_530.optimize()
    print(res.objective_value, res.status)

0.5650348730711057 optimal
0.097535323040004 optimal


In [17]:
# save medium recipe:

import os
import json 

rows = {'name': 'cdm_reuteri', 'exchanges': {}}
for r in Lreuteri_530.reactions:  # iterate over exchange reactions: 
    if len(r.metabolites)==1 and list(r.metabolites)[0].id.rsplit('_', 1)[-1] == 'e':
        if r.bounds[0] < 0: 
            rows['exchanges'][r.id] = r.lower_bound

os.makedirs('gempipe/', exist_ok=True)
with open('gempipe/cdm_reuteri.json', "w") as json_file:
    json.dump(rows, json_file)
        

In [18]:

# save the model to file. 
cobra.io.save_json_model(Lreuteri_530, 'reference/Lreuteri_530_fixed.json')
cobra.io.write_sbml_model(Lreuteri_530, 'reference/Lreuteri_530_fixed.xml')